In [ ]:
#!/usr/bin/env python
# -*- coding: UTF-8

# Parsing HTML from `wget` run!

## Initializing

In [ ]:
#import necessary libraries
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib, urllib.request
import lxml
import re, os
from glob import glob
from nltk.stem.porter import *
stemmer = PorterStemmer()

In [ ]:
#set directories and parsing keywords
data_loc = "/vol_b/data/wget/parll_wget/" #location of downloaded website data

example_page = "https://westlakecharter.com/about/"

keywords = ["values", "academics", "skills", "perseverance", 'purpose',
                       'direction', 'mission', 'vision', 'vision', 'mission', 'our purpose',
                       'our ideals', 'ideals', 'our cause', 'curriculum','curricular',
                       'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system',
                       'structure','philosophy', 'philosophical', 'beliefs', 'believe',
                       'principles', 'creed', 'credo', 'values','moral', 'history', 'our story',
                       'the story', 'school story', 'background', 'founding', 'founded',
                       'established','establishment', 'our school began', 'we began',
                       'doors opened', 'school opened']
keywords = [x.lower() for x in keywords]

mission_keywords = ['mission',' vision ', 'vision:', 'mission:', 'our purpose', 'our ideals', 'ideals:', 'our cause', 'cause:', 'goals', 'objective']
curriculum_keywords = ['curriculum', 'curricular', 'program', 'method', 'pedagogy', 'pedagogical', 'approach', 'model', 'system', 'structure']
philosophy_keywords = ['philosophy', 'philosophical', 'beliefs', 'believe', 'principles', 'creed', 'credo', 'value',  'moral', 'respect', 'excellence', 'responsibility', 'perseverance', 'inquisitive', 'joyful', 'learning', 'gratitude']
history_keywords = ['history', 'story','our story', 'the story', 'school story', 'background', 'founding', 'founded', 'established', 'establishment', 'our school began', 'we began', 'doors opened', 'school opened']

mission_keywords = set(stemmer.stem(word) for word in mission_keywords)
curriculum_keywords = set(stemmer.stem(word) for word in curriculum_keywords)
philosophy_keywords = set(stemmer.stem(word) for word in philosophy_keywords)
history_keywords = set(stemmer.stem(word) for word in history_keywords)

In [1]:
#define helper functions
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'lxml')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)  
    return u" ".join(t.strip() for t in visible_texts)


def rawwebsitetext(urlstring):
    with urllib.request.urlopen(urlstring) as url:
        s = url.read()
    string = (text_from_html(s))
    print(string)
    return re.split(r'\s{3,}', string)


def website_all_text(datalocation):
    string = ""
    for root, dirs, files in os.walk(datalocation):
        for file in files:
            if file.endswith(".txt"):
                fileloc = open(data_loc+file, "r")
                string = string + (fileloc.read())
    return string


def filterwebsite(string):
    stringList = re.split(r'\s{3,}', string)
    validcharacters = [' ', '.', '?']
    filteredtext = []  
    
    for string in stringList:
        lowercasestring = string.lower()
        for key in keywords:
            if key in lowercasestring:
                if key in lowercasestring.split(' '): #check that the word is the whole word not part of another one
                    filteredtext.extend([string.lower()])

    filteredtext = list(set(filteredtext))
    finaltext = []
    for x in filteredtext:
        finaltext.append(x.replace('\xa0', " "))
    return finaltext

In [90]:
def score_and_categorize(pagetext_list): 
    """Takes in a list of all the relevant (filtered) text from a given webpage. 
    Uses already defined keyword lists to categorize each block of text."""
    mission_list = []
    curriculum_list = []
    philosophy_list = []
    history_list = []
    general_list = []
    
    for string in pagetext_list:
        mission_score, curriculum_score, philosophy_score, history_score = 0, 0, 0, 0
        for word in mission_keywords:
            mission_score+=string.count(word)
            if 'mission' in string:
                mission_score = 2
                
        for word in curriculum_keywords:
            curriculum_score+=string.count(word)
            if 'curriculum' in string:
                curriculum_score = 2
                
        for word in philosophy_keywords:
            philosophy_score+=string.count(word)
            if 'philosophy' in string or 'value' in string:
                philosophy_score = 2
        
        for word in history_keywords:
            history_score+=string.count(word)
            if 'history' in string:
                history_score = 2
        
        if mission_score>=2:
            mission_list.append(string)
        if curriculum_score>=2:
            curriculum_list.append(string)
        if philosophy_score>=2:
            philosophy_list.append(string)
        if history_score>=2:
            history_list.append(string)
        if mission_score + curriculum_score + philosophy_score + history_score <=3:
            general_list.append(string)
        
    return {'mission': mission_list, 'curriculum' : curriculum_list, 'philosophy': philosophy_list, 'general': general_list}

In [32]:
rawwebsitetext(example_page)

                                                               Contact Us: 2680 Mabry Dr. 95835 (916) 567-5760 | admin@westlakecharter.com      Staff Login                  About   Administration & Support Staff  Employment Opportunities  Business Services  Admission & Enrollment Information  Supply Donations  Hot Lunch Payments  Hot Lunch Menu    Board   Board Policies  Board Committees  Board Documents    WCS   Infinite Campus Login  School Dismissal Manager  Ways to Westlake    Teachers  BASE  WAVE   Meetings  Contact Us  Volunteer Opportunities  Volunteer Hours  Log Volunteer Hours  Spirit Store    Calendars   17/18 School Calendar                           About          About  Aimee Wells  2013-09-18T18:36:35+00:00   About Westlake Charter Schools  Key School Features   International Focus  Thematic Curriculum  Artistic Development  Foreign Language Instruction  School-Wide Enrichment Model   Core Values   Respect  Excellence  Responsibility  Reflective  Global Perspective  Stewa

['',
 'Contact Us: 2680 Mabry Dr. 95835 (916) 567-5760 | admin@westlakecharter.com',
 'Staff Login',
 'About',
 'Administration & Support Staff  Employment Opportunities  Business Services  Admission & Enrollment Information  Supply Donations  Hot Lunch Payments  Hot Lunch Menu',
 'Board',
 'Board Policies  Board Committees  Board Documents',
 'WCS',
 'Infinite Campus Login  School Dismissal Manager  Ways to Westlake',
 'Teachers  BASE  WAVE',
 'Meetings  Contact Us  Volunteer Opportunities  Volunteer Hours  Log Volunteer Hours  Spirit Store',
 'Calendars',
 '17/18 School Calendar',
 'About',
 'About  Aimee Wells  2013-09-18T18:36:35+00:00',
 'About Westlake Charter Schools  Key School Features',
 'International Focus  Thematic Curriculum  Artistic Development  Foreign Language Instruction  School-Wide Enrichment Model',
 'Core Values',
 'Respect  Excellence  Responsibility  Reflective  Global Perspective  Stewardship  Perseverance  Inquisitive  Joyful Learning  Gratitude',
 'Calendars

In [29]:
website_all_text(data_loc)
arjuntext = website_all_text(data_loc) #Example of Arjun's scraped text - all into one block
print(arjuntext) 

                                    Explore       x:          Home    About Us    School Information    Lab Hours    Calendar    Frequently Asked Questions    Directions    Student Handbook    School Board    School Improvement Council      Contact Us    Leadership    News    Careers    Register Your Student     Follow Us      Traducciones ~ Translations ~ 번역      Richland Two Charter High School      From the News Room View All News       Fall Break Hours On November 20th and 21st, we'll close at 3 PM.
No school November 22nd through the 24th.        R2i2 is offering Dual Credit courses for the 2017-2018 School Year Earn college credit while you complete high school!        Welcome to Our New Website! Sleek. Shiny. Built for the future. Just like us.       Upcoming Events View Calendar      Connected          Richland Two Charter High School  7900 Brookmont Lane  |  Columbia, SC 29203  |  Phone: (803)-419-1348 |  Fax: (803)-935-1212  Accessibility | © 2017 Richland School District Two

In [44]:
westlaketext = "Contact Us: 2680 Mabry Dr. 95835 (916) 567-5760 | admin@westlakecharter.com      Staff Login                  About   Administration & Support Staff  Employment Opportunities  Business Services  Admission & Enrollment Information  Supply Donations  Hot Lunch Payments  Hot Lunch Menu    Board   Board Policies  Board Committees  Board Documents    WCS   Infinite Campus Login  School Dismissal Manager  Ways to Westlake    Teachers  BASE  WAVE   Meetings  Contact Us  Volunteer Opportunities  Volunteer Hours  Log Volunteer Hours  Spirit Store    Calendars   17/18 School Calendar                           About          About  Aimee Wells  2013-09-18T18:36:35+00:00   About Westlake Charter Schools  Key School Features   International Focus  Thematic Curriculum  Artistic Development  Foreign Language Instruction  School-Wide Enrichment Model   Core Values   Respect  Excellence  Responsibility  Reflective  Global Perspective  Stewardship  Perseverance  Inquisitive  Joyful Learning  Gratitude    Calendars, Schedules, Handbooks and More   Family Handbook  2017/18 School Calendar  2017/18 Bell Schedule  WCS Charter  Strategic Plan      Westlake Charter School is a K-8th grade public elementary school created by parents and educators in Natomas. We opened our doors in 2005 and continue to grow serving more and more students each year. WCS’s primary mission is to demonstrate what is possible when school and community collaborate to create inspiring adults with the academic and social-emotional readiness to lead as Global Citizens.  Westlake Charter School students are EXPLORERS! Our curriculum focuses on diversity and appreciation of different cultures, while promoting academic excellence and foreign language acquisition. We offer various specialty classes to all of our students including Art, Physical Education, and Spanish. Our middle school program focuses on Math, Science and Technology and is executed through the use of technology. Westlake Charter School has small class sizes of approximately 22-24 students in K-3rd grade and 29 students in 4th-8th grade.  First time visitors to Westlake Charter School often remark, “What a warm and inviting school!” Creating a school that is caring and open is absolutely critical in working successfully with our children and families. Students who feel safe, secure, and appreciated gain a love of learning and will be successful scholars.  Westlake Charter School’s professional staff is relentless in seeking and trying research-based best practices that engage every student. Our lessons are hands-on and differentiated to meet the diverse needs of all students. Our teachers display a never-ending commitment to improving their own expertise. Furthermore, we would not be as successful without the support and involvement of our parents. Our parent group is outstanding in helping us achieve our goals by volunteering their time in class, fundraising for our unique programs, and providing endless words of encouragement and support. Our school team is second to none!         Recent        Westlake Weekly – November 9, 2017  November 9th, 2017      Westlake Weekly – November 3, 2017  November 3rd, 2017      Westlake Weekly – October 27, 2017  October 27th, 2017      Westlake Weekly – October 20, 2017  October 20th, 2017      Westlake Weekly – October 6th, 2017  October 6th, 2017      Westlake Weekly – September 29, 2017  September 29th, 2017                     Contact Info   2680 Mabry Dr. 95835  Phone: (916) 567-5760  Fax: (916) 567-5769  Email: admin@westlakecharter.com         Copyright 2015 Westlake Charter | All Rights Reserved | Contact Us "
westlakelist = filterwebsite(westlaketext)
print(westlakelist)

['international focus  thematic curriculum  artistic development  foreign language instruction  school-wide enrichment model', 'westlake charter school is a k-8th grade public elementary school created by parents and educators in natomas. we opened our doors in 2005 and continue to grow serving more and more students each year. wcs’s primary mission is to demonstrate what is possible when school and community collaborate to create inspiring adults with the academic and social-emotional readiness to lead as global citizens.  westlake charter school students are explorers! our curriculum focuses on diversity and appreciation of different cultures, while promoting academic excellence and foreign language acquisition. we offer various specialty classes to all of our students including art, physical education, and spanish. our middle school program focuses on math, science and technology and is executed through the use of technology. westlake charter school has small class sizes of approxim

In [92]:
school_dict = score_and_categorize(westlakelist)
school_dict['philosophy']

['westlake charter school is a k-8th grade public elementary school created by parents and educators in natomas. we opened our doors in 2005 and continue to grow serving more and more students each year. wcs’s primary mission is to demonstrate what is possible when school and community collaborate to create inspiring adults with the academic and social-emotional readiness to lead as global citizens.  westlake charter school students are explorers! our curriculum focuses on diversity and appreciation of different cultures, while promoting academic excellence and foreign language acquisition. we offer various specialty classes to all of our students including art, physical education, and spanish. our middle school program focuses on math, science and technology and is executed through the use of technology. westlake charter school has small class sizes of approximately 22-24 students in k-3rd grade and 29 students in 4th-8th grade.  first time visitors to westlake charter school often re